In [0]:
import os
import tensorflow as tf
import numpy as np

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
def model_builder():
   
  weight_decay = 1e-5
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.2))

  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Conv2D(10, (1,1), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Conv2D(10,(3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
  model.add(tf.keras.layers.AveragePooling2D(pool_size=(8,8)))
#   model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Activation('softmax'))
  
  return model

In [7]:
import os

resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = model_builder()
  model.summary()
  model.compile(
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])


W0729 19:39:31.652357 140467378722688 tpu_strategy_util.py:56] TPU system %s has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.
W0729 19:39:36.815061 140467378722688 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0

In [8]:
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=100,
    steps_per_epoch=int(np.ceil(x_train.shape[0] / 400)),
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=20,verbose=1
)

W0729 19:40:14.527329 140467378722688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training_distributed.py:411: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Epoch 1/100
125/125 [==============================] - 6s 45ms/step - loss: 1.5462 - acc: 0.4880
Epoch 2/100
125/125 [==============================] - 2s 18ms/step - loss: 1.1680 - acc: 0.6489
Epoch 3/100
125/125 [==============================] - 2s 19ms/step - loss: 0.9953 - acc: 0.7128
Epoch 4/100
125/125 [==============================] - 2s 19ms/step - loss: 0.8721 - acc: 0.7521
Epoch 5/100
125/125 [==============================] - 2s 19ms/step - loss: 0.7733 - acc: 0.7836
Epoch 6/100
125/125 [==============================] - 2s 19ms/step - loss: 0.6996 - acc: 0.8019
Epoch 7/100
125/125 [==============================] - 2s 20ms/step - loss: 0.6370 - acc: 0.8194
Epoch 8/100
125/125 [==============================] - 2s 19ms/step - loss: 0.5823 - acc: 0.8356
Epoch 9/100
125/125 [==============================] - 2s 19ms/step - loss: 0.5394 - acc: 0.8467
Epoch 10/100
125/125 [==============================] - 2s 19ms/step - loss: 0.4980 - acc: 0.8589
Epoch 11/100
125/125 [=======

In [16]:
print('MAXIMUM VALIDATION ACCURACY : ', round(max(model.history.history['val_acc'])*100,3))

MAXIMUM VALIDATION ACCURACY :  87.67


## Final accuracy : ~88% with plain vanilla network and total time consumed is 3 mins 30 seconds